
*Part 2: Python for Data Analysis III*
# Data wrangling with Pandas#

In the last tutorial we got to know some useful functions and methods to import, inspect, clean and export data.

The main focus of this tutorial will be on how to **prepare data for data analysis**.



## Getting help

In this class we will not be able to cover all aspects of Python. If you want more details, you can consult, for example, the **Python Standard Library Reference** at https://docs.python.org/3/library/ or the **Language Reference** at https://docs.python.org/3/reference/. But be warned: the amount of detail in these sources can be overwhelming. For **quick and easy-to-understand overviews** of different topics see, for example, https://www.w3schools.com/python/. Here are some specific references for today's tutorial:

*  Pandas: https://www.w3schools.com/python/pandas/default.asp
*  Statsmodels: https://www.statsmodels.org/stable/user-guide.html
*  Matplotlib: https://www.w3schools.com/python/matplotlib_pyplot.asp
*  Merging: https://pandas.pydata.org/docs/user_guide/merging.html


If you get stuck or don't remember how to do something, it is usually a good idea to **Google** your problem. Python has a large (and fast-growing) community and you will probably find answers to most of your questions online (e.g. on **Stack Overflow** or in a **Youtube tutorial**).

## Getting started

For this tutorial, we will work with the following datasets:

* ``life_satisfaction_clean.csv``
* ``trust_clean.csv``
* ``real-gdp-per-capita.csv``

You can find them in the following folder: https://drive.google.com/drive/folders/1MG5FdPPx9XR2cZJmG5BqH-xq6RSQwg5J

Copy them to an appropriate folder on your computer or your Google Drive so you can follow along with the Tutorial. Let's import the modules we will use, mount our drive, change our working directory and load in the data:

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.chdir("/content/drive/MyDrive/MyData")

In [ ]:
satisfaction = pd.read_csv("life_satisfaction_clean.csv", index_col="country")
trust = pd.read_csv("trust_clean.csv", index_col="country_code")
gdp = pd.read_csv("real-gdp-per-capita.csv")

If you didn't succeed to load the files, run the following lines of code to load them:

In [ ]:
satisfaction = pd.read_csv("http://farys.org/daten/life_satisfaction_clean.csv",
                           index_col="country")

trust = pd.read_csv("http://farys.org/daten/trust_clean.csv",
                    index_col="country_code")

gdp = pd.read_csv("http://farys.org/daten/real-gdp-per-capita.csv")


## Combining datasets

When you work with data, you will often have to combine different datasets. There are four pandas functions or methods that allow you to do this:

* ``append`` method: Append rows
* ``concat`` function: Append rows or columns
* ``join`` method: Combine data on common indices
* ``merge`` function or method (both exist): Combine data on common columns or indices

In this tutorial, we will only focus on ``concat`` and ``merge``, as they allow you to do everything (and more) you can do with ``append`` and ``join``.


### Concatenating

Suppose you have your data in two different dataframes. Let's create this situation:

In [ ]:
countries1 = satisfaction.loc[["Switzerland", "Tanzania", "Peru"],
                              ["continent", "life_satisfaction"]]
countries1

In [ ]:
countries2 = satisfaction.loc[["China", "India"],
                              ["continent", "life_satisfaction"]]
countries2

Some countries are in ``countries1`` while others are in ``countries2``. How could we **combine them into a single dataframe**? The **``concat`` function** allows you to do this:

In [ ]:
countries = pd.concat([countries1, countries2])
countries

As the (first) argument, you need to **provide a list of dataframes** (or series). Then, the rows are just stacked on top of each other.




You can also use ``concat`` to combine data **columnwise**. Suppose we have the following two dataframes we would like to combine:

In [ ]:
countries_left = satisfaction.loc[["Switzerland", "Tanzania", "Peru"],
                                  ["continent", "life_satisfaction"]]
countries_left

In [ ]:
countries_right = satisfaction.loc[["Switzerland", "Tanzania", "Peru"],
                                   ["gni_per_capita", "population"]]
countries_right

To **concatenate dataframes columnwise, we need to set the ``axis`` parameter to 1**:

In [ ]:
countries = pd.concat([countries_left, countries_right], axis=1)
countries

Now the columns from ``countries_left`` and the columns from ``countries_right`` are combined into one dataframe.

### Merging (one-to-one)

Now suppose you would like to combine the life satisfaction data (``satisfaction``) with the trust data (``trust``). Both datasets contain information on several countries and you would like to combine the data in a way such that the countries are matched. How could this be done? You could try to concatenate the two datasets, but this is very risky. It will not work if the number  or order of the countries differs and you may get things mixed up. It would be better to **combine the data based on the index or the values of some column(s)** (e.g. the country code).

The **``merge`` function or method** (`merge` exists as a function and as a method) allows you to do this. The basic syntax is

```python
new_df = pd.merge(left_df, right_df, ...) # function
new_df = left_df.merge(right_df, ...)     # method
```

where `left_df` and `right_df` are the two dataframes you want to combine. In the examples below, we will use the method syntax.

Let's take a look at our two dataframes:

In [ ]:
satisfaction.head()

In [ ]:
trust.head(7)

Before we can merge the dataframes, we need to decide **on what we want to merge them**. We can merge dataframes **by the index or by column values**. The merge method has the following parameters to specify this:
* **``left_index``**: Set to ``True`` if the left dataframe should be **merged on the index**
* **``right_index``**: Set to ``True`` if the right dataframe should be **merged on the index**
* **``left_on``**: Specify what **column(s)** from the left dataframe should be taken for the merge
* **``right_on``**: Specify what **column(s)** from the left dataframe should be taken for the merge
* **``on``**: Specify the name of the **column(s)** for the merge if the column name(s) is/are the **same in both datasets**

Let's try **merge by the index of both dataframes**:

In [ ]:
df = satisfaction.merge(trust,
                        left_index=True,   # use index from left dataframe for merge
                        right_index=True)  # use index from right dataframe for merge
df.head(2)

What happened? Since the index is not the same between the two datasets (country names in `satisfaction` and country codes in `trust`), there were no matches and we created an empty dataset.

In many cases, you will have to **merge on column values** instead of the index. In our case, we could merge on the country code. The country code is in column ``code`` in ``satisfaction`` (the left dataframe) and in the index in ``trust``(the right dataframe). So we have to merge on column `code` for the left dataframe and on the index for the right dataframe:

In [ ]:
df = satisfaction.merge(trust,
                        left_on="code",
                        right_index=True)

df.head(2)

> <font color = 4e1585>SIDENOTE: Even though `country_code` is the index in the right dataframe and not a column, we can treat it in `merge()` as if it was a column; `merge()` is smart enough to understand what we mean. That is, we can also merge the datasets as follows:
>
>
>```python
df = satisfaction.merge(trust,
                        left_on="code",
                        right_on="country_code")
```
>
><font color = 4e1585>A slight difference is that in the second variant of the code the index is not passed on to the new dataframe (which makes sense because the index is not the same in the two datasets).
>
>
><font color = 4e1585>Moreover, you can also **merge on several columns**. For example, if you worry that different countries may have the same country code, you could additionally match on continent. Observations are then only merged if both the country code and the continent are the same:
>
> ```python
> df = satisfaction.merge(trust,
                        left_on=["code", "continent"],
                        right_on=["country_code", "continent"])
```



Let's think a bit more closely about what happened when we merged the ``satisfaction`` and the ``trust`` dataset. Consider the length of the two datasets:

In [ ]:
print(len(satisfaction))
print(len(trust))

Clearly, there must be countries that are in ``satisfaction`` but not in ``trust``. There may also be countries that are in ``trust`` but not in ``satisfaction``. How does the ``merge`` method handle these cases?

By default, ``merge`` performs an ``inner join`` -- only observations that are in both dataframes are included. This is why our merged dataframe ``df`` has fewer observations than ``satisfaction`` and ``trust``:

In [ ]:
print(len(df))



The**``how`` parameter allows you to specify what observations should be included** in the merged dataframe. There are 4 possibilities:

* **``inner`` join**: Only observations that are in *both* dataframes are included. This is the default.
* **``outer`` join**: *All* observations are included.
* **``left`` join**: All observations from the *left* dataset (i.e. ``satisfaction``) are included.
* **``right`` join**: All observations from the *right* dataset (i.e. ``trust``) are included.

Let's perform an outer join, so we do not loose any observations:

In [ ]:
df = satisfaction.merge(trust,
                        how="outer",  # set type of join
                        left_on="code",
                        right_on="country_code",
                        indicator=True)  # add a column with merging information
df.head(3)

Since we set the `indicator` parameter to `True`, a column named ``_merge`` was added to the dataframe that contains information on the source of an observation. Let's take a look at it:

In [ ]:
df["_merge"].value_counts()

As we can see, most of the observations only appeared in the ``satisfaction`` dataframe, some appeared in both dataframes and very few only appeared in the ``trust`` dataframe.

> <font color = 4e1585>SIDENOTE: Merging datasets from different sources often requires a lot of tedious data cleaning. For example, if you merge on country names, the spelling of these names may differ across datasets meaning that these countries will not be merged. It may be a good idea to start with an outer merge and inspect the observations that appear only in one of the datasets. If available, (standardized) codes are usually much better for merging than names.  

---

>  <font color='teal'> **In-class exercise**:
Consider the following two dataframes:  

In [ ]:
my_data1 = satisfaction.loc[["Switzerland", "Tanzania", "Peru", "China"],
                            ["code", "life_satisfaction"]].reset_index()

my_data2 = satisfaction.loc[["Tanzania", "India", "Peru", "China"],
                            ["code", "gni_per_capita", "workhours"]].reset_index()
my_data2.loc[3, "country"] = " People's Republic of China"

In [ ]:
my_data1

In [ ]:
my_data2

>  <font color='teal'> How could you combine them into one keeping all observations from ``my_data1``?




---



### Many-to-one merging

Up to now, we have only conducted **one-to-one joins**. Each observation in your left dataset was merged to one observation from the right dataset (or to none, if none was found). Sometimes you will also want to conduct **many-to-one** joins. This means that several observations from your left dataset will be merged to the same observation in your right dataset. Consider the following two example datasets:

In [ ]:
df1 = pd.DataFrame({"year": [2000, 2019, 2000, 2019],
                    "gdp_per_cap": [37868, 81994, 411, 1122]},
                   index=["Switzerland", "Switzerland", "Tanzania", "Tanzania"])
df1

In [ ]:
df2 = pd.DataFrame({"continent": ["Europe", "Africa"]},
                   index=["Switzerland", "Tanzania"])
df2

How could you merge these two datasets? Performing many-to-one joins with pandas is straightforward, as **pandas will infer automatically what kind of join to perform**:

In [ ]:
df1.merge(df2, left_index=True, right_index=True)

Before you merge, always be sure that you are perfectly aware of the structure (especially duplicates) of your data! Consider the following example:

In [ ]:
df_dup = pd.DataFrame({"continent": ["Europe", "Africa"]},
                      index=["Switzerland", "Switzerland"])  # Switzerland ist duplicated
df1.merge(df_dup, left_index=True, right_index=True,
          how="left")  # introduces duplicates on the left side as well!

df_dup.index.is_unique

> <font color = 4e1585>SIDENOTE: The merge method also allows you to perform **many-to-many** or **one-to-many** joins. If you would like know more about different types of joins, see, for example: https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html

## Grouping data

Our life satisfaction data contains a variable classifying the countries by income level. Suppose you would like to look at the **average life satisfaction by income group**. How could we do this? One way would be to filter the data and print the result for each country group:

In [ ]:
satisfaction

In [ ]:
print(satisfaction.loc[satisfaction["income_level"] == "High income",
                       "life_satisfaction"].mean())
print(satisfaction.loc[satisfaction["income_level"] == "Upper middle income",
                       "life_satisfaction"].mean())
# etc.

This can get very tedious, especially if you have many categories/groups. The **``groupby()`` method allows you to split your data into groups and to easily compute summary statistics (or do operations) for each group**. Let's group our data by income level:

In [ ]:
grouped_df = satisfaction.groupby("income_level")
print(grouped_df)

Using the **``get_group()`` method**, we can retrieve all observations belonging to a particular group:

In [ ]:
grouped_df.get_group("Low income")

Now we can compute the mean (or other summary statistics) for each group:

In [ ]:
grouped_df["life_satisfaction"].mean()

We can also use ``groupby()`` to create an aggregated dataframe:

In [ ]:
ls_byincome = grouped_df["life_satisfaction"].agg(["mean", "median", "count"])
ls_byincome

If you want to have the group label as a data column rather than the index, you can add `reset_index()`:

In [ ]:
def mean2(x):
    return x.mean()


grouped_df["life_satisfaction"].agg(["mean", "median", "count", mean2]).reset_index()

><font color = 4e1585> SIDENOTE: You can also group your data by several columns (e.g. ``df.groupby(["income_level", "continent"])``. If you want to know more about grouping data, see, for example:
* https://pandas.pydata.org/docs/user_guide/groupby.html

---

>  <font color='teal'> **In-class exercise**:
You would like to know how many hours people work in a typical country on each continent. Use the ``groupby()`` method compute the median of `workhours` by continent!

>  <font color='teal'> Now create a summary dataframe called ``workhours_by_continent`` with the median of `workhours` and the number of countries by continent.



---



## Reshaping data

Two-dimensional data can often be displayed in two different formats: **long** or **wide**. Consider the following dataframe:

In [ ]:
df_long = pd.DataFrame({"country": ["Switzerland", "Switzerland", "Tanzania", "Tanzania"],
                        "year": [2000, 2019, 2000, 2019],
                        "gdp_per_cap": [37868, 81994, 411, 1122]})
df_long

We call this dataframe **long** because there are multiple/repeated observations (gdp per year) per observational unit (country). Another representation of the data would be to have each country just once and have additional columns which describe the gdp at different years:

In [ ]:
df_wide = df_long.pivot(index="country", columns="year", values="gdp_per_cap")
print(df_wide)
print(df_wide.columns)
print(df_wide.index)

That does not look very clean as we get hierarchical columns. We can solve that by resetting the index:

In [ ]:
df_wide = df_wide.reset_index()
df_wide

Further, you might consider to first change the values of ``year`` in a way that they are proper column names, e.g. ``"gdp_2000"`` instead of ``2000``.

In [ ]:
df_long["colnames"] = "gdp_" + df_long["year"].astype(str)  # <- not ideal
df_wide = df_long.pivot(index="country", columns="colnames",
                        values="gdp_per_cap").reset_index()
df_wide

In the other direction you can also reshape from a wide to a long format using ``melt()`` or ``wide_to_long()``:

In [ ]:
df_long2 = pd.melt(df_wide, id_vars="country",
                   value_vars=["gdp_2000", "gdp_2019"],
                   var_name="year",
                   value_name="gdp_per_cap")
df_long2["year"] = (df_long2["year"].str.split("_").str[1]
                    .astype(int))  # change values from e.g. "gdp_2000" to 2000 (integer)
df_long2

Reshaping tasks can often be pretty complicated and confusing. You can read additional information in the pandas reference: <https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html>. If you run into problems with reshaping: start small! Use tiny examples, try your code, and carefully inspect what it does.

---

>  <font color='teal'> **In-class exercise**:
Generate the following dataframe:

In [ ]:
df = pd.DataFrame({"name": ["John Smith", "Jane Doe", "Mary Johnson"],
                   "treatmentA": [5, 10, 8], "treatmentB": [12, 10, 9]})
df

---

>  <font color='teal'> Reshape the dataset in a way that you have a column ``treatment`` with possible values ``A`` and ``B`` and the measured value as ``value``. Assign the result to a new dataframe ``df_long``.

In [ ]:
# We might want to clean the new column a bit:

---

>  <font color='teal'> Now, reshape the ``df_long`` dataframe back to a wide format.